In [5]:
import nltk
from nltk.stem import WordNetLemmatizer
import json
import pickle
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout
import random
from keras.optimizers import SGD

In [6]:
lemmatizer = WordNetLemmatizer()

words = []
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('data.json').read()
intents = json.loads(data_file)

for intent in intents['intents']:
    for pattern in intent['patterns']:

        # tokenize each word
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        # add documents in the corpus
        documents.append((w, intent['tag']))

        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

# lemmaztize and lower each word and remove duplicates
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))
# sort classes
classes = sorted(list(set(classes)))
# documents = combination between patterns and intents
print(len(documents), "documents")
# classes = intents
print(len(classes), "classes", classes)
# wrds = all wrds, vocabulary
print(len(words), "unique lemmatized wrds", words)

pickle.dump(words, open('words.pkl', 'wb'))
pickle.dump(classes, open('classes.pkl', 'wb'))

134 documents
19 classes ['Airbus production', 'Create DN', 'Create WO', 'ODD', 'Products and Services Airbus', 'SOI', 'Status', 'about', 'check stock', 'contact SO', 'goodbye', 'greetings', 'help', 'push a need', 'reserve a need', 'thanks', 'track a PO', 'track a WO', 'what is Airbus']
108 unique lemmatized wrds ['&', "'s", ',', 'a', 'ability', 'about', 'airbus', 'and', 'anyone', 'appreciate', 'are', 'assist', 'awesome', 'be', 'bye', 'can', 'chatting', 'check', 'client', 'contact', 'could', 'create', 'creation', 'day', 'define', 'demand', 'dn', 'do', 'email', 'evening', 'feature', 'for', 'get', 'give', 'going', 'good', 'goodbye', 'great', 'greeting', 'hello', 'help', 'helpful', 'hey', 'hi', 'how', 'i', 'is', 'it', 'item', 'know', 'later', 'me', 'mean', 'morning', 'my', 'name', 'need', 'next', 'nice', 'odd', 'of', 'offer', 'offered', 'one', 'order', 'pattern', 'po', 'procedure', 'process', 'product', 'production', 'provide', 'purchase', 'push', 'request', 'reserve', 'see', 'send', 'ser

In [7]:
# create our training data
training = []
# create an empty array for our output
output_empty = [0] * len(classes)
# training set, bag of wrds for each sentence
for doc in documents:
    # initialize our bag of wrds
    bag = []
    # list of tokenized wrds for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related wrds
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of wrds array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)
# create train and test lists. X - patterns, Y - intents
train_x = list(training[:, 0])
train_y = list(training[:, 1])
print("Training data created")

Training data created


C:\Users\Brahim Laghzaoui\AppData\Local\Temp\ipykernel_12252\1200432142.py:24: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training = np.array(training)


In [8]:
# Create model - 3 layers. First layer 128 neurons,
# second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
modell = Sequential()
modell.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
modell.add(Dropout(0.5))
modell.add(Dense(64, activation='relu'))
modell.add(Dropout(0.5))
modell.add(Dense(len(train_y[0]), activation='softmax'))

# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this modelll
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
modell.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

# fitting and saving the model
hist = modell.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
modell.save('chatbot_model.h5', hist)

print("model created")


Epoch 1/200
27/27 [==============================] - 1s 2ms/step - loss: 2.9804 - accuracy: 0.0597 
Epoch 2/200
27/27 [==============================] - 0s 3ms/step - loss: 2.8516 - accuracy: 0.1418
Epoch 3/200
27/27 [==============================] - 0s 3ms/step - loss: 2.7434 - accuracy: 0.1642
Epoch 4/200
27/27 [==============================] - 0s 2ms/step - loss: 2.6118 - accuracy: 0.2313
Epoch 5/200
27/27 [==============================] - 0s 3ms/step - loss: 2.4840 - accuracy: 0.2537
Epoch 6/200
27/27 [==============================] - 0s 3ms/step - loss: 2.1993 - accuracy: 0.3358
Epoch 7/200
27/27 [==============================] - 0s 3ms/step - loss: 2.0962 - accuracy: 0.4030
Epoch 8/200
27/27 [==============================] - 0s 3ms/step - loss: 1.8897 - accuracy: 0.4328
Epoch 9/200
27/27 [==============================] - 0s 3ms/step - loss: 1.6945 - accuracy: 0.5746
Epoch 10/200
27/27 [==============================] - 0s 3ms/step - loss: 1.5288 - accuracy: 0.5149
Epoch 11